In [20]:
import data
import data_custom
from vgg import VGG
import torch

from torch.utils.mobile_optimizer import optimize_for_mobile

In [2]:
_, _, fer2013 = data.get_dataloaders('./Datasets/FER2013', 8)
_, _, fer_plus = data.get_dataloaders('./Datasets/FERplus', 8)
_, _, custom_face_data = data_custom.get_dataloaders('./Datasets/CustomFaceData', 8)

In [5]:
archs = {
    "VGG_11": ((1, 64), (1, 128), (2, 256), (2, 512), (2, 512)),
    "VGG_16": ((2, 64), (2, 128), (3, 256), (3, 512), (3, 512)),
}


In [234]:
net = VGG(arch=archs["VGG_16"])
path = f'./checkpoints/VGG_16_FER2013/epoch_150'
checkpoint = torch.load(path)
net.load_state_dict(checkpoint["params"])

e:\Anaconda\TestML\Lib\site-packages\torch\nn\modules\lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


<All keys matched successfully>

In [236]:
# initialise lazy network

inputs = next(iter(fer2013))[0]
# fuse crops and batchsize
original_shape = inputs.shape
inputs = inputs.view(-1, *inputs.shape[-3:])

# forward
outputs = net(inputs)

In [237]:
model_scripted = torch.jit.script(net.eval())
model_scripted.save("./Trained Models/VGG16-FER2013.pt")
